## Introducción al Web Scraping

### ¿Qué es web scraping?
Scraping es una técnica que permite extraer información de una página web. Mediante esta técnica podemos obtener información de una página web y utilizarla para nuestros propios fines, aunque la página no proporcione una API para acceder a la información.

### Casos de uso comunes:
- Análisis de competencia
- Monitoreo de precios
- Investigación de mercados
- Agregación de noticias
- Machine learning (obtención de datos de entrenamiento)

### Consideraciones éticas y legales:
- Respetar robots.txt (ejemplo: https://www.clarin.com/robots.txt)
- No sobrecargar servidores 
- Verificar términos de servicio (ejemplo donde está prohibido el scraping: https://www.inmuebles.clarin.com/terminoscondiciones)
  
Todo lo que vamos a ver en esta clase es para fines educativos y siempre se debe respetar los términos de uso de las páginas web. 

## Scraping Básico con requests y BeautifulSoup

Vamos a comenzar con un ejemplo práctico que consiste en extraer titulares de noticias desde una página web. Para ello, utilizaremos dos librerías: requests y BeautifulSoup.  
  
La librería requests nos facilita la tarea de realizar requests, es decir, nos permite conectarnos a una página web y obtener su contenido, tal como lo haría un navegador. 

Por otro lado, BeautifulSoup es una herramienta para el análisis y la manipulación de documentos HTML y XML. Gracias a ella, podemos "entender" la estructura del HTML descargado y extraer de manera sencilla los elementos que nos interesan, como por ejemplo los títulos de las noticias.

In [ ]:
# Importar librerías necesarias
import requests
from bs4 import BeautifulSoup

# URL de ejemplo - BBC News
url = 'https://www.bbc.com/news'

# Hacer la petición HTTP
response = requests.get(url)

# Verificar el estado de la respuesta
print(f"Código de estado: {response.status_code}")

# Analizar el contenido HTML
if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Encontrar todos los elementos h2 (títulos)
    headlines = soup.find_all('h2')

    # Mostrar los primeros 10 titulares
    print("\nTítulos encontrados en BBC News:")
    for idx, headline in enumerate(headlines[:10], 1): 
        print(f"{idx}. {headline.get_text(strip=True)}")
else:
    print(f"Error al acceder a la página: {response.status_code}")

### Ejercicio práctico
Modifica el código anterior para extraer los enlaces (href) de cada titular

## Ejemplo 2: Scraping de productos en Mercado Libre

Vamos a extraer información de productos de Mercado Libre como ejemplo. 
(Mercado libre tiene una API, pero vamos a usar scraping para este ejemplo)

In [ ]:
# Scraping de productos en Mercado Libre
import pandas as pd

repuestos = "https://listado.mercadolibre.com.ar/suspension-honda-civic-2008?sb=all_mercadolibre#D[A:suspension%20honda%20civic%202008]"

response = requests.get(repuestos)

if response.status_code != 200:
    print(f"Error al acceder a la página: {response.status_code}")
else:
    print("Acceso exitoso a la página")
soup = BeautifulSoup(response.text, 'html.parser')

# Selector específico para productos (puede cambiar con el tiempo. Eso es lo malo del scraping)
items = soup.find_all('div', class_='poly-card poly-card--list poly-card--large poly-card--CORE')

listado = []
for item in items:
    titulo = item.find("a").text
    price = item.find_all("div", class_="poly-component__price")
    price = price[0].text.split("$")[1]
    listado.append({"titulo": titulo, "precio": price})

# Crear DataFrame
df = pd.DataFrame(listado)


In [ ]:
## veamos precios: que pasa con el contenido?
df.head(10)

### Extracción de Tablas HTML

Muchos sitios muestran datos en tablas. Por ejemplo, el BCRA, muchas páginas de finanzas, del gobierno, incluso wikipedia.  
Cuando tenemos tablas, pandas nos facilita la vida.  

In [ ]:
import pandas as pd

url = "https://www.bcra.gob.ar/SistemasFinancierosYdePagos/Proveedores-servicios-de-billeteras-digitales-Interoperables.asp"
body = requests.get(url, verify=False)

tables = pd.read_html(body.text)

if tables:
    df = tables[0]
    ## limpiamos columnas
    df.columns = df.columns.get_level_values(1)
else:
    print("No se encontraron tablas en la página")

In [ ]:
df

## Introducción a Selenium


Para páginas con contenido dinámico (JavaScript) necesitamos Selenium.

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time


## hay que instalar un driver. Una forma facil es usar un ejecutable en la misma carpeta

driver = webdriver.Chrome(executable_path="./chromedriver")

driver.get("https://www.google.com")
time.sleep(3)
driver.quit()

In [7]:
## otra forma es usar el driver manager

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager


# Opcion 1: especificar una version que sabemos que existe
# driver = webdriver.Chrome(service=Service(ChromeDriverManager("129.0.6494.0").install()))

# Opcion 2: Usar la ultima version 

##from webdriver_manager.core.utils import ChromeType
## pero para eso hay que instalar 
##pip install --upgrade webdriver-manager
#driver = webdriver.Chrome(service=Service(ChromeDriverManager(chrome_type=ChromeType.CHROMIUM).install()))



In [40]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time

# Configurar el driver
driver = webdriver.Chrome()

In [31]:


# Ejemplo: Extraer comentarios de YouTube (que se cargan con JavaScript)
url = "https://www.bancoprovincia.com.ar/cuentadni/contenidos/cdniIndividuos"
driver.get(url)
time.sleep(5)

In [32]:
## podemos ver el html
html = driver.page_source
html

'<html lang="es" class="s1-chat-initilized Chrome 130 desktop Linux landscape"><head><script type="text/javascript" id="www-widgetapi-script" src="https://www.youtube.com/s/player/22f02d3d/www-widgetapi.vflset/www-widgetapi.js" async=""></script><script type="text/javascript" async="" src="https://www.youtube.com/iframe_api"></script><script async="" src="https://www.googletagmanager.com/gtm.js?id=GTM-5S8SFMX&amp;gtm_auth=&amp;gtm_preview=&amp;gtm_cookies_win=x"></script><script>\n      (function(w,d,s,l,i){w[l]=w[l]||[];\n        w[l].push({\'gtm.start\': new Date().getTime(),event:\'gtm.js\', });\n        var f=d.getElementsByTagName(s)[0],j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';\n        j.async=true;j.src=\'https://www.googletagmanager.com/gtm.js?id=\'+i+dl+\'&gtm_auth=&gtm_preview=&gtm_cookies_win=x\';\n        f.parentNode.insertBefore(j,f);\n      })(window,document,\'script\',\'dataLayer\',\'GTM-5S8SFMX\');</script><script type="text/javascript" async="" src="ht

In [33]:
## selenium, ademas de leer el html, nos permite interactuar con la pagina
## por ejemplo, podemos hacer click en un boton
## buscar el boton con clase "callmodal8"

boton = driver.find_element(By.CLASS_NAME, "callmodal8")
boton.click()

In [34]:
## y podemos buscar el boton con clase "close" para cerrar el modal
modal = driver.find_element(By.CLASS_NAME, "modal8")

close = modal.find_element(By.CLASS_NAME, "close")
close.click()


In [35]:
## ademas de manejarnos por las clases, podemos manejarnos por los ids.
## por ejemplo, los matecitos de la pagina tienen id s1_chat_img

matecitos = driver.find_element(By.ID, "s1_chat_img")
matecitos.click()

In [36]:
## otra forma de hacer click es usando xpath
## el xpath es una ruta que nos permite llegar a un elemento de la pagina
## ahora veamos en el navegador como obtener el xpath de un elemento

xpath_transporte = '//*[@id="icono_transporte_img"]'

transporte = driver.find_element(By.XPATH, xpath_transporte)
transporte.click()

In [56]:
## No solamente podemos hacer click, sino que podemos escribir texto
## por ejemplo, vamos a https://www.bancoprovincia.com.ar/formulario/SemiPresencial/PrestamoPersonal/?origenSolicitud=Web

driver.get("https://www.bancoprovincia.com.ar/formulario/SemiPresencial/PrestamoPersonal/?origenSolicitud=Web")

## ahora vamos a escribir el nombre
## vamos a buscar los inputs por name

nombre = driver.find_element(By.NAME, "nombreyApellido")
nombre.send_keys("Juan Perez")

cuit = driver.find_element(By.NAME, "cuit")
cuit.send_keys("20241111111")

## buscar el de email por name=email
email = driver.find_element(By.NAME, "email")
email.send_keys("juanperez@gmail.com")


In [ ]:
## vamos a saltearnos un par de inputs y usar los selectores 


opcion = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div[2]/div[2]/div/div/form/div[6]/div/div/select/option[3]')
opcion.click()

## veamos ubicacion por Xpath

ubicacion = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div/div[2]/div[2]/div/div/form/div[7]/div[1]/div[1]/select/option[2]')
ubicacion.click()


In [62]:
## Esta página tiene un recaptcha, por lo que si hacemos click en el boton de enviar, posiblemente falle.
## hay metodos para bypassar el recaptcha, pero no vamos a verlos en este curso.

### Ejercicio Selenium  
  
Entrar a la pagina https://www.bancoprovincia.com.ar/adhesioncomercios/INDEX?utm_medium=Organico&utm_source=Comercios_Lapos&utm_campaign=Adhesion&utm_content=Adhesion_comercios
  
1. Completar todos los campos  
2. Hacer click en el boton de "Acepto que mis datos de contacto..."
3. NO hacer click en el boton de "Enviar", no queremos llenar de spam.  


## Ejercicio final de Scraping  
  
Utilizando selenium o requests + beautifulsoup, ir a la pagina de banco provincia:   
https://www.bancoprovincia.com.ar/web/lineas_evolucion_cap_trabajo  
  
1. Obtener las urls de los links que aparecen en la pagina
2. Visitar cada una de las urls y extraer:  
  Destinatarias  
  Destino de los fondos  
  Plazo y forma de pago  
  Garantías  
  Tasa de interés  

3. Guardar todo en un dataframe